# Object Detectiuon Module Using Pre-Trained Networks for Security Systems
This notebook shows an object detection module using pre-trained networks such as **ResNet50** for feature extraction and **YOLOv3** for object detection. The module is designed to detect hazardous objects (e.g., knives, guns, fire) that can be integrated into a security system for real-time monitorign or image analysis.

## Import required packages

In [ ]:
# import numpy as np
import tensorflow as tf
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.applications.resnet50 import preprocess_input
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.models import Model
# import matplotlib.pyplot as plt
# import cv2

